# Warehouse Woes

## Part 1

In [72]:
def parse_input(sample=None):

	def read_input(sample=sample, file_path="Warehouse_Woes.txt", mode="r"):
		if sample is not None:
			file_path = f"sample{sample}.txt"

		with open(file_path, mode) as file:
			return file.read()

	data = read_input(sample)

	data = data.split("\n\n")

	grid = [list(row) for row in data[0].split("\n")]

	instructions = "".join( row for row in data[1].split("\n"))

	return grid, instructions

In [73]:
def find_start(grid, start_symbol="@"):
	for i, row in enumerate(grid):
		for j, cell in enumerate(row):
			if cell == start_symbol:
				return i, j
			
	return "Not Found"

In [74]:
def print_grid(grid):
	for row in grid:
		print("".join(row))

In [75]:
def gps_score(grid, x_mult=100, y_mult=1):
	score = 0
	for i, row in enumerate(grid):
		for j, cell in enumerate(row):
			if cell == "O":
				score += i * x_mult + j * y_mult

	return score

In [76]:
def push_boxes(grid, position, direction):
	x, y = position
	dx, dy = direction

	i = 0
	while True:
		i += 1
		nx, ny = x+i*dx, y+i*dy

		# If we reach a wall, do nothing, no movement
		if grid[nx][ny] == "#":
			return x, y
		
		# If we reach a free space:
		# 	Move box to the free space
		# 	Move robot to next position in the direction of movement
		elif grid[nx][ny] == ".":
			# Move box to the free spot
			grid[nx][ny] = "O"
			# Move robot to the first box position
			grid[x+dx][y+dy] = "@"
			# Remove the robot from previous position
			grid[x][y] = "."
			# Update the position of the robot
			x, y = x+dx, y+dy
			return x, y

		# Else, we reached a box. Do nothing, keep looking ahead 
		# for a free space or a wall
		elif grid[nx][ny] == "O":
			continue 

		else:
			print("Error: Invalid cell")

In [77]:
def simulate_movement(grid, instructions, start_pos, verbose=False):
	x, y = start_pos
	# directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
	directions_dict = {
	">": (0, 1),
	"v": (1, 0),
	"<": (0, -1),
	"^": (-1, 0)
}
	
	for instruction in instructions:
		dx, dy = directions_dict[instruction]
		
		if verbose:
			print(f"Robot at {x}, {y} moving {instruction}")

		if grid[x + dx][y + dy] == "#":
			pass
		elif grid[x + dx][y + dy] == ".":
			grid[x + dx][y + dy] = "@"
			grid[x][y] = "."
			x += dx
			y += dy
		else:
			x, y = push_boxes(grid, (x, y), (dx, dy))

		if verbose:
			print_grid(grid)
			print()


	score = gps_score(grid)

	return grid, score

In [78]:
def part1(sample=None, verbose=False):
	grid, instructions = parse_input(sample)
	start_pos = find_start(grid)
	grid, score = simulate_movement(grid, instructions, start_pos, verbose)

	return score

In [79]:
print(f"Small sample: {part1(sample=2)}")
print(f"Large sample: {part1(sample=1)}")
print(f"\nSolution: {part1()}")

Small sample: 2028
Large sample: 10092

Solution: 1421727


## Part 2

In [80]:
def transform_grid(grid):
	new_grid = []
	for row in grid:
		new_row = ""
		for cell in row:
			if cell == "#":
				new_row += "##"
			elif cell == "O":
				new_row += "[]"
			elif cell == "@":
				new_row += "@."
			else:
				new_row += ".."
		new_grid.append(list(new_row))

	return new_grid

In [132]:
def is_valid_move(grid, positions, direction):

	current_layer = []
	valid_layer = True



	for position in positions:
		x, y = position
		dx, dy = direction
		nx, ny = x + dx, y + dy

		if grid[nx][ny] == "#":
			return False
		
		# If moving up or down
		if direction == (1, 0) or direction == (-1, 0):
			if grid[nx][ny] == "[":
				current_layer.extend([(nx, ny), (nx, ny+1)])
			elif grid[nx][ny] == "]":
				current_layer.extend([(nx, ny-1), (nx, ny)])

		# If moving left or right
		else:
			if grid[nx][ny] == "[":
				current_layer.extend([(nx, ny+1)])
			elif grid[nx][ny] == "]":
				current_layer.extend([(nx, ny-1)])


	if len(current_layer) != 0:
		valid_layer = is_valid_move(grid, current_layer, direction)

	if not valid_layer:
		return False
	
	# Update positions
	for position in positions:
		x, y = position
		dx, dy = direction
		nx, ny = x + dx, y + dy

		if grid[x][y] == "@":
			grid[nx][ny] = "@"
			grid[x][y] = "."
		
		elif grid[x][y] == "[":
			grid[nx][ny:ny+2] = "[]"
			grid[x][y:y+2] = ".."
		
		elif grid[x][y] == "]":
			grid[nx][ny-1:ny+1] = "[]"
			grid[x][y-1:y+1] = ".."


	x, y = positions[0]
	nx, ny = x + dx, y + dy
	if len(positions) == 1 and grid[nx][ny] == "@":
		return nx, ny
	else:
		return True

In [133]:
def simulate_movement_v2(grid, instructions, start_pos, verbose=False):
	x, y = start_pos
	# directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
	directions_dict = {
		">": (0, 1),
		"v": (1, 0),
		"<": (0, -1),
		"^": (-1, 0)
	}
	
	if verbose:
		print("Initial Grid")
		print_grid(grid)
		print()

	for instruction in instructions:
		dx, dy = directions_dict[instruction]
		
		if verbose:
			print(f"Moving {instruction}")

		nx, ny = x + dx, y + dy
		
		if grid[nx][ny] == "#":
			pass
		elif grid[nx][ny] == ".":
			grid[nx][ny] = "@"
			grid[x][y] = "."
			x += dx
			y += dy

		else:
			status = is_valid_move(grid, [(x, y)], (dx, dy))
			if status == False:
				pass
			else:
				x, y = status

		if verbose:
			print_grid(grid)
			print()

	return grid

In [134]:
data, instructions = parse_input(sample=3)
grid = transform_grid(data)
start_pos = find_start(grid, start_symbol="@")

grid_updated = simulate_movement_v2(grid, instructions, start_pos, verbose=True)

Initial Grid
################
##..@.[][]....##
##............##
################

Moving >
################
##...@[][]....##
##............##
################

Moving >
################
##....@.].]...##
##............##
################

Moving >
################
##.....@].]...##
##............##
################

Moving >


RecursionError: maximum recursion depth exceeded

In [96]:
print_grid(grid_updated)

####################
##[]..[]......[][]##
##[]...........[].##
##[]..........[][]##
##[].......[]...[]##
##..##............##
##..@.............##
##.......[]...[][]##
##.....[][].......##
####################
